In [ ]:
import os
import sys
import xarray as xr
import cftime
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs


In [ ]:
dir_trendy_v8_S2  = '/Data/SLAND_Trendy-v8_S2_LatLon/'
dir_trendy_v8_S3  = '/Data/SLAND_Trendy-v8_S3_LatLon/'
dir_trendy_v9_S2  = '/Data/SLAND_Trendy-v9_S2_LatLon/'
dir_trendy_v9_S3  = '/Data/SLAND_Trendy-v9_S3_LatLon/'
dir_trendy_v10_S2 = '/Data/SLAND_Trendy-v10_S2_LatLon/'
dir_trendy_v10_S3 = '/Data/SLAND_Trendy-v10_S3_LatLon/'
dir_grids         = '/Data/grids/'
dir_forest        = '/Data/forest_masks/'
dir_fig           = '/Figures/'


## Calculate forest fraction weight from Hansen 2013 compared to pre-industrial Trendy data

In [ ]:
#Define models
models_all = ['CABLE-POP', 'CLASSIC', 'CLM5.0', 'ISAM', 'ISBA-CTRIP', 'JSBACH', 'JULES-ES-1.1', 'LPJ-GUESS', 'LPJwsl', 'LPX-Bern', 'OCN', 'ORCHIDEEv3', 'SDGVM', 'VISIT', 'YIBs']

#Select all natural land or forests only
selection = 'NaturalLandCoverFraction'
selection = 'ForestFraction'

#Skip CABLE-POP as there is a problem with the natural land cover fraction (I did not investigate this further, since it only affects a Supplementary Figure)
if selection=='NaturalLandCoverFraction':
    models = models_all[1::]
else:
    models = models_all

#Define maximum values for weighting factor
max_values = [2, 10, 50, 100]

if selection=='ForestFraction':
    var_name     = 'forest_fraction'
    var_name_out = 'w_FF'
    fname_out = 'Weights_ForestFraction'
elif selection=='NaturalLandCoverFraction':
    var_name     = 'natural_land_cover_fraction'
    var_name_out = 'w_NLCF'
    fname_out = 'Weights_NaturalLandCoverFraction'

#Loop over files/models
create = 1
area_coll = []
for ii, model in enumerate(models):
    
    
    #Define correct Trendy folder
    if model=='CABLE-POP':
        dir_trendy_S2 = dir_trendy_v8_S2
        dir_trendy_S3 = dir_trendy_v8_S3
    elif model=='CLM5.0':
        dir_trendy_S2 = dir_trendy_v9_S2
        dir_trendy_S3 = dir_trendy_v8_S3
    elif model in ['CLASSIC-N', 'SDGVM']:
        dir_trendy_S2 = dir_trendy_v10_S2
        dir_trendy_S3 = dir_trendy_v10_S3
    else:
        dir_trendy_S2 = dir_trendy_v9_S2
        dir_trendy_S3 = dir_trendy_v9_S3

    #Use forest fraction of JULES-ES-1.0 for JULES-ES-1.1 (since it is not available for the latter)
    if model=='JULES-ES-1.1':  model_read = 'JULES-ES-1.0'
    else:                      model_read = model
        
    #Get files that contain forest fraction for Trendy models
    files_S2 = sorted([file for file in os.listdir(dir_trendy_S2) if model_read + '_' in file and selection + '.nc' in file])
    files_S3 = sorted([file for file in os.listdir(dir_trendy_S3) if model_read + '_' in file and selection + '.nc' in file])
    if len(files_S2)!=1:  sys.exit('File name not unique')
    if len(files_S3)!=1:  sys.exit('File name not unique')
    
    #Read data
    fname_S2 = dir_trendy_S2 + files_S2[0]
    fname_S3 = dir_trendy_S3 + files_S3[0]
    data_S2 = xr.open_dataset(fname_S2)
    data_S3 = xr.open_dataset(fname_S3)
    
    data_S2 = data_S2[var_name].to_dataset(name=var_name)
    data_S3 = data_S3[var_name].to_dataset(name=var_name)
    
    #Calculate average over last 20 years if time variable exists
    if 'time' in data_S2.dims:  data_S2 = data_S2.isel(time=slice(-20, None)).mean('time')
    if 'time' in data_S3.dims:  data_S3 = data_S3.isel(time=slice(-20, None)).mean('time')        
    
    #Calculate fraction as fraction of S3 to S2 simulations
    for_weight = data_S3[var_name] / data_S2[var_name]

    #Only allow weighting factors larger than 1 for land grid points with at least 0.1% forest cover
    for_weight = xr.where(data_S2[var_name]>0.001, for_weight, 1)

    #Replace NaNs bei 0
    for_weight = for_weight.fillna(0)
    
    #Convert to data set
    for_weight = for_weight.to_dataset(name=var_name_out)

    #Define output file name
    fname_weight = dir_forest + fname_out + '_DGVMs-S2-S3_grid_' + model + '.nc'
    if os.path.exists(fname_weight):  os.remove(fname_weight)
    
    #Save data (and regrid Trendy-v9 data to Trendy-v10 grid for ORCHIDEEv3)
    if model!='ORCHIDEEv3':
            
        #Save in file
        for_weight.to_netcdf(fname_weight)
        
    else:
        
        #Define temporary file names
        fname_grid_tmp = dir_forest + 'grid_xy_' + model + '_tmp'
        fname_tmp1     = dir_forest + 'data_' + model + '_tmp1.nc'
        fname_tmp2     = dir_forest + 'data_' + model + '_tmp2.nc'
        if os.path.exists(fname_grid_tmp):  os.remove(fname_grid_tmp)
        if os.path.exists(fname_tmp1):      os.remove(fname_tmp1)
        if os.path.exists(fname_tmp2):      os.remove(fname_tmp2)
        
        #Save data in NetCDF and set grid
        for_weight.to_netcdf(fname_tmp1)
        os.system('cdo -s griddes -selvar,' + var_name + ' ' + fname_S2 + ' > ' + fname_grid_tmp)
        os.system('cdo -s setgrid,' + fname_grid_tmp + ' ' + fname_tmp1 + ' ' + fname_tmp2)
        
        #Conservatively regrid weights
        file_grid = dir_grids + 'grid_xy_' + model
        os.system('cdo -s remapcon,' + file_grid + " " + fname_tmp2 + " " + fname_weight)

        #Remove temporary files
        os.remove(fname_grid_tmp)
        os.remove(fname_tmp1)
        os.remove(fname_tmp2)        
